### Import Libraries

In [1]:
### Import Librariesfrom selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd

import time
from datetime import date, datetime, timedelta

import re

### Give Path and Initialize Chrome Driver

In [37]:
path = "https://qaranjobs.com/"

#chrome_driver = './chromedriver.exe'

time.sleep(2)

driver = webdriver.Chrome()
driver.get(path)

### Scroll height bar one screen height each time and sleep to wait in between

In [38]:
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

In [ ]:
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
#         time.sleep(2)
        load_more_btn = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div/div[2]/div[1]/article/div/div[2]/div/a")))
        load_more_btn.click();
# driver.quit()

### Download whole scrolled page source locally

In [40]:
import codecs
with open('page_source.html','w',encoding='UTF-8') as f:
    f.write(driver.page_source)
    f.close()

### Get Specific Column data for each job title

In [42]:
def extract_from_url(url, job_data):
    print(url)
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, "html.parser")
    details=soup.find('div',class_='job_description').text
    match = re.search(r'(?P<educational_requirement>.*)', details)
    print(match.group('educational_requirement'))
#     regex = r"degree([\w\s,]+)\."
#     row=re.findall(regex,details)
#     if len(row)==0:
#         regex = r"master([\w\s,]+)\."
#         row=re.findall(regex,details)
#     print(row)

In [43]:
urls = []
jobs=[]
with open('page_source.html','r',encoding='UTF-8') as f:
    page_source=f.read()
    f.close()
soup = BeautifulSoup(page_source, "html.parser")
# print(soup)
job_listing=soup.find('ul',class_="job_listings")
for job in job_listing.find_all("li",class_='job_listing'):
    job_data={}
    job_data["title"]=job.find('div',class_='position').text.split('–')[0]
    if(len(job.find('div',class_='company').text))>2:
        job_data["company"]=job.find('div',class_='company').text
    else:
        job_data["company"]=job.find('div',class_='position').text.split('–')[0]
    job_data["location"]=job.find('div',class_='location').text
    job_data["posted"]=job.find('li',class_='date').text.split('on')[1]
    link = job.find('a').attrs['href']
    urls.append(link)
    job_data["url"]=link
    jobs.append(job_data)
#     extract_from_url(link,job_data)
#     print(job_data['title'])
#     time.sleep(2)
# driver.quit()

In [30]:
print(jobs)

[{'title': '\nSomali jobs: The Federal Republic of Somalia (5 positions) are closing today, March 31st, 2024.\n\nFederal Republic of Somalia \n', 'company': '\nFederal Republic of Somalia ', 'location': '\n\t\t\tSomalia\t\t', 'posted': ' March 31, 2024', 'url': 'https://qaranjobs.com/job/somali-jobs-the-federal-republic-of-somalia-5-positions-are-closing-today-march-31st-2024/'}, {'title': '\nSomali jobs at Save the Children (6 positions)\n\nSave the Children \n', 'company': '\nSave the Children ', 'location': '\n\t\t\tSomalia\t\t', 'posted': ' March 30, 2024', 'url': 'https://qaranjobs.com/job/somali-jobs-at-save-the-children-6-positions/'}, {'title': '\nSomali jobs at UNICEF (3 Positions)\n\nUnited Nations Children’s Fund (UNICEF) \n', 'company': '\nUnited Nations Children’s Fund (UNICEF) ', 'location': '\n\t\t\tSomalia\t\t', 'posted': ' March 24, 2024', 'url': 'https://qaranjobs.com/?post_type=job_listing&p=30592'}, {'title': '\nJobs at Somali Red Crescent Society (SRCS) (4 Position

In [46]:
df = pd.DataFrame(jobs)

### Format Data by labelling it as stem job or not and remove delimiters

In [45]:
import re
def split_string(x):
    return x.split('\n')[1]
def split_tab(x):
    new_string = re.sub(r'\t', '', x)
    return new_string
def categorize_stem_job(row):
    job=row['title']
    stem=0
#     print(job)
    if job.strip() in stem_jobs:
        stem=1
    else:
        stem=0
#     print(stem)
    return stem

In [47]:
cols_format=['title','company','location']
for col in cols_format:
    df[col]=df[col].apply(split_string)
df['stem']=df.apply(lambda row:categorize_stem_job(row),axis=1)
df['location']=df['location'].apply(split_tab)

In [48]:
df

,title,company,location,posted,url,stem
0,Somali jobs: The Federal Republic of Somalia (...,Federal Republic of Somalia,Somalia,"March 31, 2024",https://qaranjobs.com/job/somali-jobs-the-fede...,0
1,Somali jobs at Save the Children (6 positions),Save the Children,Somalia,"March 30, 2024",https://qaranjobs.com/job/somali-jobs-at-save-...,0
2,Somali jobs at UNICEF (3 Positions),United Nations Children’s Fund (UNICEF),Somalia,"March 24, 2024",https://qaranjobs.com/?post_type=job_listing&p...,0
3,Jobs at Somali Red Crescent Society (SRCS) (4 ...,Ururka Bisha Cas (SRCS),Somalia,"March 23, 2024",https://qaranjobs.com/?post_type=job_listing&p...,0
4,Finance Assistance,Amtel,Somalia,"March 20, 2024",https://qaranjobs.com/job/finance-assistance-a...,0
...,...,...,...,...,...,...
5755,Child Protection Information Management Officer,IRC,"Awbare, Shedder and Kebribeyah, Somali Regiona...","July 27, 2022",https://qaranjobs.com/job/child-protection-inf...,0
5756,Terms of Reference (ToR) for Open tender for P...,Candlelight,"Buhodle, Somaliland","July 27, 2022",https://qaranjobs.com/job/terms-of-reference-t...,0
5757,Saferty And Security Officer,Polish Humanitarian Action (PAH),"Mogadishu, Somalia","July 27, 2022",https://qaranjobs.com/job/saferty-and-security...,0
5758,Finance Officer,NRC,"Kismayo, Somalia","July 27, 2022",https://qaranjobs.com/job/finance-officer-kism...,0


### Save Cleaned Data to CSV

In [35]:
df.to_csv('Qaran_jobs_raw_2Years.csv', index=False, header=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     4320 non-null   object
 1   company   4320 non-null   object
 2   location  4320 non-null   object
 3   posted    4320 non-null   object
 4   url       4320 non-null   object
 5   stem      4320 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 202.6+ KB
